This block just imports the potential_fitting module.

In [1]:
import potential_fitting

In [2]:
from potential_fitting.database import Database
with Database() as db:
    db.annihilate("confirm")

Now define filepaths as variables so we don't have to keep retyping strings.
Make sure you have nothing of value in any of these files, as all files will be overwritten by certain steps of the workflow except settings, unopt_geo, and database.

In [3]:
NO3_settings = "NO3-monomer.ini"
H2O_settings = "H2Omonomer.ini"
dimer_settings = "dimer.ini"
unopt_NO3 = "NO3-.xyz"
unopt_H2O = "H2O.xyz"
opt_NO3 = "NO3.opt.xyz"
opt_H2O = "H2O.opt.xyz"
training_configs = "training_configs.xyz"
test_configs = "test_configs.xyz"
database = "database.db"
training_set = "training_set.xyz"
test_set = "test_set.xyz"
poly_in = "A1B3_C1D2.in"
molecule_in = "A1B3_C1D2"
poly_directory = "poly"
config = "config.ini"
ttm_directory = "ttm"
fit_directory = "fit"
fitted_code = "fitted"

method = "HF"
basis = "STO-3G"
cp = True

client_name = "someclient"

Optimizes the single geometry specified in the file located at the path specifed by unopt_geo and writes the optimized geometry to the file path specified by opt geo. This may take a while depending on what method/basis you use.

In [8]:
potential_fitting.optimize_geometry(NO3_settings, unopt_NO3, opt_NO3)
potential_fitting.optimize_geometry(H2O_settings, unopt_H2O, opt_H2O)

Beginning geometry optimization using psi4 of NO3- with HF/STO-3G.
Optimizer: Optimization complete!
Completed geometry optimization.
Beginning geometry optimization using psi4 of H2O with HF/STO-3G.
Optimizer: Optimization complete!
Completed geometry optimization.


Generates 50 rigid 2b configurations from the optimized geometry specified by opt_geo into the xyz file specified by configs

In [4]:
potential_fitting.generate_2b_configurations(dimer_settings, opt_NO3, opt_H2O, 2, training_configs, min_distance = 2, max_distance = 10, min_inter_distance = 0)
potential_fitting.generate_2b_configurations(dimer_settings, opt_NO3, opt_H2O, 2, test_configs, min_distance = 2, max_distance = 10, min_inter_distance = 0)

File ./training_configs.xyz already exists, moving existing ./training_configs.xyz to ./training_configs.xyz.backup-1 to make way for new file.
Generated 2 configurations
File ./test_configs.xyz already exists, moving existing ./test_configs.xyz to ./test_configs.xyz.backup-1 to make way for new file.
Generated 2 configurations


This puts the configurations located in the file specified by configs into the database specified by database. It will initialize the database if it does not already exist, if it exists, new configurations will be added from the configs file. Model, basis, and cp to be used to calculate energy are also set at this time as specifed in the settings file. Note that cp (counterpoise correction) has no effect for 1b systems. The second call adds the optimized geometry to the database.

In [4]:
potential_fitting.init_database(dimer_settings, training_configs, method, basis, cp, "training")
potential_fitting.init_database(NO3_settings, opt_NO3, method, basis, cp, "training", optimized = True)
potential_fitting.init_database(H2O_settings, opt_H2O, method, basis, cp, "training", optimized = True)

potential_fitting.init_database(dimer_settings, test_configs, method, basis, cp, "test")
potential_fitting.init_database(NO3_settings, opt_NO3, method, basis, cp, "test", optimized = True)
potential_fitting.init_database(H2O_settings, opt_H2O, method, basis, cp, "test", optimized = True)

Initializing database from xyz file training_configs.xyz directory into database.
Initializing of database successful.
Initializing database from xyz file NO3.opt.xyz directory into database.
Initializing of database successful.
Initializing database from xyz file H2O.opt.xyz directory into database.
Initializing of database successful.
Initializing database from xyz file test_configs.xyz directory into database.
Initializing of database successful.
Initializing database from xyz file NO3.opt.xyz directory into database.
Initializing of database successful.
Initializing database from xyz file H2O.opt.xyz directory into database.
Initializing of database successful.


Loops through every uncalculated energy in the database and calculates it. This will take a while depending on what method/basis you use.

In [ ]:
potential_fitting.fill_database(dimer_settings, client_name)

Filling database.
     1     2     3     4     5

Generates a training set in the file specified by training_set from all "CO2-CO2" (molecules with two fragments each with name "CO2") energies in the database, additional arguments can be added to prune by method, basis, cp, or tag. If your database contains energies computed with a variety of methods/bases, it is probably a good idea to use only one of them in your training set at a time. The training set contains each configuration's deformation energy. The optimized geometry will have energy 0.

In [ ]:
potential_fitting.generate_2b_training_set(dimer_settings, training_set, "H2O-NO3-", "H2O", "NO3-", method, basis, cp, "training")
potential_fitting.generate_2b_training_set(dimer_settings, test_set, "H2O-NO3-", "H2O", "NO3-", method, basis, cp, "test")

This call generates a polynomial input file based on the symmetry of molecule "CO2" in the database, it will be generated in the current working directory

In [ ]:
potential_fitting.generate_poly_input_from_database(dimer_settings, database, "NO3--H2O", ".")

Generate polynomials of degree 3 based on the polynomial input file as specified by poly_in (should be the file created by the above call, will be symmetry as specified in settings file + ".in" for 1b) in the directory specified by poly_directory

In [ ]:
potential_fitting.generate_polynomials(dimer_settings, poly_in, 3, poly_directory)

Convert the polynomial maple files in poly_directory into cpp files

In [ ]:
potential_fitting.execute_maple(dimer_settings, poly_directory)

Does a qchem calculation to generate the config file containing charges, polarizabilities, etc in the file specified by config based on the molecule symmetry specified by molecule_in (same as polynomial input but without the .in) and the optimized geometry as specified by opt_geo. This will take a while, but should not be effected by your method/basis.

In [ ]:
potential_fitting.generate_fit_config(dimer_settings, molecule_in, config, opt_NO3, opt_H2O)

Generates the 2b TTM fit code of degree 3 in fit_directory using the config file specified by config, and the polynomial files in poly_directory

In [ ]:
potential_fitting.generate_2b_ttm_fit_code(dimer_settings, config, molecule_in, ttm_directory)

Compile the fit code in fit_directory

In [ ]:
potential_fitting.compile_fit_code(dimer_settings, ttm_directory)

Generate a fitted .nc file at location "fit.nc" using the fit code in fit_directory, the training set as specified by training_set, and the fitcode as specified by fit_directory + "/fit-1b".

In [ ]:
potential_fitting.fit_2b_ttm_training_set(dimer_settings, ttm_directory + "/fit-2b-ttm", training_set, ttm_directory, config)

In [ ]:
potential_fitting.generate_2b_fit_code(dimer_settings, config, poly_in, poly_directory, 2, fit_directory)

In [ ]:
potential_fitting.compile_fit_code(dimer_settings, fit_directory)

In [ ]:
potential_fitting.fit_2b_training_set(dimer_settings, fit_directory + "/fit-2b", training_set, fit_directory, "fitted.nc")

In [ ]:
%matplotlib inline
potential_fitting.fitting.fit_visualizer.compare_energies("ttm/eval-2b-ttm", "ttm/ttm-params.txt", "fit/eval-2b", "fitted.nc", database, "NO3-", "H2O", "%", "%", "%", "test")